# download rankings
This code scrapes and downloads subject-specific QS World University Rankings. It saves HTML pages that another ipynb file converts to a nice table. 

Methodology for the rankings:
- https://www.topuniversities.com/subject-rankings/methodology
- http://www.iu.qs.com/university-rankings/subject-tables/

Disclaimers: 
- Rankings are dubious measures of quality and they get used in unethical ways. They can be useful for studying prestiege and inequality tho, e.g. some folks have shown they correlate strongly with faculty hiring.
- I don't own this data. QS makes it free & public. I just offer this to help people put it in a nice format for statistical use. 
    - If you represent QS and you want this taken down, just send me an email.
- They might change their website and break this code. No promises I'll update the code if that happens. 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time
from tqdm import tqdm

### First, get the list of subject links

In [ ]:
driver = webdriver.Firefox()

In [ ]:
base_url = 'https://www.topuniversities.com'

In [ ]:
driver.get("https://www.topuniversities.com/university-rankings/university-subject-rankings/2021/arts-humanities#university-rankings-indicators")


In [ ]:
#select the subject list
cn = 'f-field-2 margin-right-24px subject width268px position-relative qsrankingsubject-realdiv'
cn = '.'+cn.replace(' ', '.')
elem = driver.find_element_by_css_selector(cn)

#get the list of subjects
#cn = 'menu transition hidden'
cn = 'menu'
cn = '.'+cn.replace(' ', '.')
elem = elem.find_element_by_css_selector(cn)

#get the links for subjects
tmp = elem.find_elements_by_class_name('item')
subject_links = []
for l in tmp:
    subject_links.append(l.get_attribute('data-value'))
    
subject_links

In [ ]:
# if you need to re-do some of the subjects
manual_subject_links = [
    '/university-rankings/university-subject-rankings/2021/sociology',
    '/university-rankings/university-subject-rankings/2021/social-sciences-management',
]

### Second, scrape the actual pages.

In [ ]:
driver = webdriver.Firefox()

In [ ]:
#for l in tqdm(manual_subject_links):

for l in tqdm(subject_links):
    driver.get(base_url+l)
    time.sleep(1)
    
    #get field names
    field = l.split('/')[-1]
    print(field)
    
    # tab to detailed ranking data
    cn = 'nav-link quicktabs-tab-rankings_tabs-1 add_padding quick-navsj'
    cn = '.'+cn.replace(' ', '.')
    driver.find_element_by_css_selector(cn).click()
    time.sleep(2)
    
    # more results per page
    try:
        element = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".dropdown.icon"))
        )
    except:
        print('error')
        driver.quit()
    
    cn = 'row mt-3 perpagedata_d-none_js results_perpage_data'
    cn = '.'+cn.replace(' ', '.')
    elem = driver.find_element_by_css_selector(cn)

    cn = 'dropdown icon'
    cn = '.'+cn.replace(' ', '.')
    elem.find_element_by_css_selector(cn).click()
    time.sleep(1)

    elems = elem.find_elements_by_class_name('item')
    for e in elems:
        if e.text == '100':
            e.click()
            time.sleep(5)          
            
    # tab through pages
    more_pages = True
    page = 1
    while more_pages:
        # save page
        
        try:
            element = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CLASS_NAME, "uni-link"))
            )
        except:
            print('error')
            driver.quit()
        
        time.sleep(5)
        with open('rankings/source_'+field+'_'+str(page)+'.html', 'w') as o:
            o.write(driver.page_source)
        
        #advance to next page
        try:
            cn = 'page-link next'
            cn = '.'+cn.replace(' ', '.')
            driver.find_element_by_css_selector(cn).click()
            page += 1
        except NoSuchElementException:
            break